# 训练配置文件

In [1]:
%%writefile ./mmsegmentation-030/080100_upernet_convnext_l_fold0_bs8_2.5e-4_train512_test512_noflip_adamwsw_ce10_config.py

# work_dir = f'/home/br/workspace/GRIC/output/080414'

num_classes = 2
out_channels = 1
threshold = 0.8
class_weight = [10]
fold = 0 

norm_cfg = dict(type='BN', requires_grad=True) # 分割框架通常使用 SyncBN
custom_imports = dict(imports='mmcls.models', allow_failed_imports=False)

# checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/convnext/downstream/convnext-xlarge_3rdparty_in21k_20220301-08aa5ddc.pth' # xlarge

checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/convnext/downstream/convnext-large_3rdparty_in21k_20220301-e6e0ea0a.pth' # large

# checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/convnext/downstream/convnext-base_3rdparty_in21k_20220301-262fd037.pth' # base

model = dict(
    type = 'EncoderDecoder',
    pretrained = None,

    backbone = dict(
        type='mmcls.ConvNeXt',
        arch='large', #### base large xlarge
        out_indices=[0, 1, 2, 3],
        drop_path_rate=0.4,
        layer_scale_init_value=1.0,
        gap_before_final_norm=False,
        init_cfg=dict(
            type='Pretrained', checkpoint=checkpoint_file,
            prefix='backbone.')
            ),

    decode_head = dict(
        type='UPerHead', # 解码头(decode head)的类别。
        # in_channels=[256, 512, 1024, 2048], # xlarge
        in_channels=[192, 384, 768, 1536], # large
        # in_channels=[128, 256, 512, 1024], # base
        in_index=[0, 1, 2, 3], # 选择特征图的索引
        pool_scales=(1, 2, 3, 6),
        channels=512, # 解码头中间态(intermediate)的通道数。
        dropout_ratio=0.1,  # 进入最后分类层(classification layer)之前的 dropout 比例。
        num_classes=num_classes, # 分割前景的种类数目。 (仅二分类问题中为2，前景和背景)
        out_channels=out_channels, # 辅助头输出的通道数。
        threshold=threshold, # 二分类时预测的阈值，仅在推理时有效。
        norm_cfg=norm_cfg,  # 归一化层的配置项。
        align_corners=False, # 解码里调整大小(resize)的 align_corners 参数。
        loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0, class_weight=class_weight)
        # loss_decode=dict(type='FocalLoss', use_sigmoid=True, gamma=2.0, alpha=[1.0], loss_weight=1.0, class_weight=class_weight),
        # loss_decode=dict(type='DiceLoss', loss_weight=1.0, class_weight=[10, 0.1]), # , class_weight=[10]
        ),

    auxiliary_head = dict(
        type='FCNHead', # 辅助头(auxiliary head)的种类。可用选项请参考 mmseg/models/decode_heads。
        # in_channels=1024, # xlarge
        in_channels=768, # large
        # in_channels=512, # base
        in_index=2, # 被选择的特征图(feature map)的索引。
        channels=256, # 辅助头中间态(intermediate)的通道数。
        num_convs=1, # FCNHead 里卷积(convs)的数目. 辅助头里通常为1。
        concat_input=False, # 在分类层(classification layer)之前是否连接(concat)输入和卷积的输出。
        dropout_ratio=0.1, # 进入最后分类层(classification layer)之前的 dropout 比例。
        num_classes=num_classes, # 分割前景的种类数目。 (仅二分类问题中为2，前景和背景)
        out_channels=out_channels, # 辅助头输出的通道数。
        threshold=threshold, # 二分类时预测的阈值，仅在推理时有效。
        norm_cfg=norm_cfg,  # 归一化层的配置项。
        align_corners=False, # 解码里调整大小(resize)的 align_corners 参数。
        loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=0.4, class_weight=class_weight)
        # loss_decode=dict(type='FocalLoss', use_sigmoid=True, gamma=2.0, alpha=[1.0], loss_weight=0.4, class_weight=class_weight),
        # loss_decode=dict(type='DiceLoss', loss_weight=0.4, class_weight=[10, 0.1]), # , class_weight=[10]
        ),

    # model training and testing settings
    train_cfg = dict(),
    test_cfg = dict(mode='whole'), # 测试模式， 选项是 'whole' 和 'sliding'. 'whole': 整张图像全卷积(fully-convolutional)测试。 'sliding': 图像上做滑动裁剪窗口(sliding crop window)。
)


# dataset settings
dataset_type = 'CustomDataset' # 数据集类型 CustomDataset NpyDataset
data_root = '../input/mmseg_data/' # 数据的根路径

classes = ['BG', 'contrails']
palette = [[0,0,0], [255,0,0]]

img_norm_cfg = dict(
    # mean=[0,0,0], std=[1,1,1], to_rgb=True
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True
)

albu_train_transforms = [
    dict(type='RandomBrightnessContrast', p=0.5),
]


train_img_size = [(512,512)]
test_img_size = [(512,512)]

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    # 多尺度训练
    dict(type='Resize', 
    img_scale=train_img_size,
    ),
    ####
    dict(type='RandomFlip', prob=0.00000001, direction='horizontal'),
    # dict(type='RandomFlip', prob=0.5, direction='vertical'),
    # dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **img_norm_cfg),
    # dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']), # 决定数据里哪些键被传递到分割器里的流程。
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=test_img_size,
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False, # 测试时是否翻转图像。
        transforms=[
            dict(type='Resize', keep_ratio=True), # 使用改变图像大小的数据增广
            dict(type='RandomFlip'), # 考虑到 RandomFlip 已经被添加到流程里，当 flip=False 时它将不被使用。
            dict(type='Normalize', **img_norm_cfg),  # 归一化配置项，值来自 img_norm_cfg。
            # dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
            dict(type='ImageToTensor', keys=['img']), # 将图像转为张量
            dict(type='Collect', keys=['img']), # 收集测试时必须的键的收集流程。
        ])
]

img_suffix = '.npy'
fore_str = "_fore" # _fore
data = dict(
    samples_per_gpu = 8,
    workers_per_gpu = 4,

    train = dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=img_suffix,
        seg_map_suffix='.png',
        split=f"splits/fold_{fold}{fore_str}.txt",
        classes=classes,
        palette=palette,
        pipeline=train_pipeline,
        ),

    val = dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=img_suffix,
        seg_map_suffix='.png',
        split=f"splits/holdout_{fold}.txt",
        classes=classes,
        palette=palette,
        pipeline=test_pipeline),

    test = dict(
        type=dataset_type,
        data_root=data_root,
        test_mode=True,
        img_dir='images',
        ann_dir='labels',
        img_suffix=img_suffix,
        seg_map_suffix='.png',

        classes=classes,
        palette=palette,
        pipeline=test_pipeline))



# yapf:disable
log_config = dict(
    interval = 100, # 打印日志的间隔
    hooks = [ # 训练期间执行的钩子
        dict(type='TextLoggerHook', by_epoch=False),
        # dict(type='MMSegWandbHook', by_epoch=False, # 还支持 Wandb 记录器，它需要安装 `wandb`。
        #      init_kwargs={'entity': "OpenMMLab", # 用于登录wandb的实体
        #                   'project': "mmseg", # WandB中的项目名称
        #                   'config': cfg_dict}), # 检查 https://docs.wandb.ai/ref/python/init 以获取更多初始化参数
    ])
# yapf:enable
dist_params = dict(backend='nccl') # 用于设置分布式训练的参数，端口也同样可被设置。
log_level = 'INFO' # 日志的级别。
load_from = None  # 从一个给定路径里加载模型作为预训练模型，它并不会消耗训练时间。
resume_from = None # 从给定路径里恢复检查点(checkpoints)，训练模式将从检查点保存的轮次开始恢复训练。
workflow = [('train', 1)] # runner 的工作流程。 [('train', 1)] 意思是只有一个工作流程而且工作流程 'train' 仅执行一次。
cudnn_benchmark = True  # 是否是使用 cudnn_benchmark 去加速，它对于固定输入大小的可以提高训练速度。

# optimizer
optimizer = dict(
    constructor='LearningRateDecayOptimizerConstructor',
    type='AdamW',
    lr=2.5e-4, ####
    betas=(0.9, 0.999),
    weight_decay=0.05,
    paramwise_cfg={
        'decay_rate': 0.9,
        'decay_type': 'stage_wise',
        'num_layers': 12
    }
    )


optimizer_config = dict(type='Fp16OptimizerHook', loss_scale='dynamic')
# learning policy
lr_config = dict(
    policy='poly', # 调度流程的策略，同样支持 Step, CosineAnnealing, Cyclic 等.
    warmup='linear',
    warmup_iters=1500,
    warmup_ratio=1e-6,
    power=1.0, # 多项式衰减 (polynomial decay) 的幂。
    min_lr=0.0, # 用来稳定训练的最小学习率。
    by_epoch=False)

# runtime settings
runner = dict(type='EpochBasedRunner', max_epochs=30)
checkpoint_config = dict(by_epoch=True, interval=1, save_last=True, max_keep_ckpts=15)
evaluation = dict(interval=1, metric=['mDice'], pre_eval=True)

fp16 = dict()

Overwriting ./mmsegmentation-030/080900_upernet_convnext_l_fold0_bs8_2.5e-4_train512_test512_noflip_adamwsw_ce10_config.py


# 开始训练

In [ ]:
%cd ./mmsegmentation-030/
!python ./tools/train.py ./080812_upernet_convnext_l_fold1_bs8_2.5e-4_train512_test512_adamwsw_ce10_config.py --gpu-id 0